In [1]:
import pandas as pd  # For data handling and manipulation

In [3]:
# Set the path to your input file
input_file = 'linkedin_jobs.csv'  # or 'linkedin_jobs.json'

# Load the data
if input_file.endswith('.csv'):
    df = pd.read_csv(input_file)
elif input_file.endswith('.json'):
    df = pd.read_json(input_file, lines=True)
else:
    raise ValueError("Unsupported file format. Use CSV or JSON.")

# Show the first few rows to verify
df.head()


,jobid,job_title,job_level,job_url,industry,job_description,job_salary
201,Data Engineer,Mid,https://www.linkedin.com/jobs/view/3928327681/,Technology,Build data pipelines and manage data platforms,$105,0
202,Senior Data Engineer,Senior,https://www.linkedin.com/jobs/view/3927117499/,Finance,Lead ETL processes and mentor junior engineers,$125,0
203,Data Platform Engineer,Mid,https://www.linkedin.com/jobs/view/3930213714/,Technology,Develop scalable data infrastructure and services,$110,0
204,Cloud Data Engineer,Mid,https://www.linkedin.com/jobs/view/3929893687/,Cloud Computing,Manage cloud data workflows and storage systems,$108,0
201,Data Engineer,Mid,https://www.linkedin.com/jobs/view/3928327681/,Technology,Build data pipelines and manage data platforms,$105,0


In [5]:
# Set the column name to identify duplicates (change if your column name is different)
job_id_column = 'jobid'

# Check if the column exists
if job_id_column not in df.columns:
    raise ValueError(f"Column '{job_id_column}' not found in data.")

# Get number of rows before deduplication
before = len(df)

# Drop duplicates
df_clean = df.drop_duplicates(subset=job_id_column)

# Get number of rows after deduplication
after = len(df_clean)

print(f"Removed {before - after} duplicates. Remaining rows: {after}")


Removed 2 duplicates. Remaining rows: 8


In [7]:
# Define output file name
output_file = 'linkedin_jobs_deduplicated.csv'

# Save to CSV
df_clean.to_csv(output_file, index=False)

print(f"Cleaned data saved to: {output_file}")


Cleaned data saved to: linkedin_jobs_deduplicated.csv


In [9]:
df_clean.head() #preview cleaned data


,jobid,job_title,job_level,job_url,industry,job_description,job_salary
201,Data Engineer,Mid,https://www.linkedin.com/jobs/view/3928327681/,Technology,Build data pipelines and manage data platforms,$105,0
202,Senior Data Engineer,Senior,https://www.linkedin.com/jobs/view/3927117499/,Finance,Lead ETL processes and mentor junior engineers,$125,0
203,Data Platform Engineer,Mid,https://www.linkedin.com/jobs/view/3930213714/,Technology,Develop scalable data infrastructure and services,$110,0
204,Cloud Data Engineer,Mid,https://www.linkedin.com/jobs/view/3929893687/,Cloud Computing,Manage cloud data workflows and storage systems,$108,0
205,ETL Developer,Entry,https://www.linkedin.com/jobs/view/3930306452/,Retail,Design and develop ETL mappings and data valid...,$90,0


In [11]:
import pandas as pd
import requests
from tqdm import tqdm  # For progress bar

In [13]:
# Update with your actual file path
input_file = 'linkedin_jobs_deduplicated.csv'
df = pd.read_csv(input_file)

# Show the first few rows
df.head()

,jobid,job_title,job_level,job_url,industry,job_description,job_salary
0,Data Engineer,Mid,https://www.linkedin.com/jobs/view/3928327681/,Technology,Build data pipelines and manage data platforms,$105,0
1,Senior Data Engineer,Senior,https://www.linkedin.com/jobs/view/3927117499/,Finance,Lead ETL processes and mentor junior engineers,$125,0
2,Data Platform Engineer,Mid,https://www.linkedin.com/jobs/view/3930213714/,Technology,Develop scalable data infrastructure and services,$110,0
3,Cloud Data Engineer,Mid,https://www.linkedin.com/jobs/view/3929893687/,Cloud Computing,Manage cloud data workflows and storage systems,$108,0
4,ETL Developer,Entry,https://www.linkedin.com/jobs/view/3930306452/,Retail,Design and develop ETL mappings and data valid...,$90,0


In [15]:
def is_job_url_valid(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }

        response = requests.get(url, headers=headers, timeout=10, allow_redirects=True)
        status = response.status_code

        # 1. Check HTTP status codes that imply job is gone
        if status in [404, 410]:
            return False

        # 2. Check for soft-deleted jobs or redirect to search
        invalid_phrases = [
            "This job is no longer accepting applications",
            "Job has expired",
            "Page not found",
            "Search for more jobs",
            "We can’t find this job",
            "This job has closed",
            "The job you were looking for was not found"
        ]
        if any(phrase in response.text for phrase in invalid_phrases):
            return False

        if response.url != url:
            # LinkedIn redirected to another page
            if "/jobs/search/" in response.url or "/jobs/" not in response.url:
                return False

        # 3. If status is 200 and content looks OK
        return True

    except requests.RequestException:
        return False

In [17]:
# Add a new column 'is_valid' by checking each job_url
tqdm.pandas(desc="Checking job URLs")
df['is_valid'] = df['job_url'].progress_apply(is_job_url_valid)

# Filter out invalid URLs
df_clean = df[df['is_valid'] == True].drop(columns=['is_valid'])

# Show how many were removed
print(f"Removed {len(df) - len(df_clean)} expired or broken job URLs.")

Checking job URLs: 100%|████████████████████████| 8/8 [00:00<00:00, 2054.39it/s]

Removed 8 expired or broken job URLs.


In [19]:
output_file = 'linkedin_jobs_NoExpired.csv'
df_clean.to_csv(output_file, index=False)
print(f"Cleaned data saved to: {output_file}")

Cleaned data saved to: linkedin_jobs_NoExpired.csv


In [21]:
df_clean.head() #preview cleaned data

,jobid,job_title,job_level,job_url,industry,job_description,job_salary
